In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pt
import plotly.express as px

In [3]:
def remove_collinear_features(x, threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model 
        to generalize and improves the interpretability of the model.

    Inputs: 
        x: features dataframe
        threshold: features with correlations greater than this value are removed

    Output: 
        dataframe that contains only the non-highly-collinear features
    '''

    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                #print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns=drops)
    print('Removed Columns {}'.format(drops))
    return x




In [4]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

In [5]:
train.isna().sum().sum()

6762701

In [6]:
train.dropna(inplace = True)
train.isna().sum().sum()

0

In [7]:
feats = train[[str(col) for col in train.columns if 'feature' in col]]

In [8]:
X = pd.concat([remove_collinear_features(feats, 0.9), train['weight'], train['ts_id']], axis = 1)

y = pd.Series([1 if x > 0 else 0 for x in train['resp']])

Removed Columns {'feature_68', 'feature_126', 'feature_35', 'feature_38', 'feature_100', 'feature_128', 'feature_66', 'feature_28', 'feature_125', 'feature_113', 'feature_40', 'feature_110', 'feature_107', 'feature_61', 'feature_23', 'feature_122', 'feature_108', 'feature_69', 'feature_36', 'feature_88', 'feature_26', 'feature_57', 'feature_119', 'feature_4', 'feature_25', 'feature_76', 'feature_6', 'feature_114', 'feature_58', 'feature_127', 'feature_101', 'feature_129', 'feature_33', 'feature_124', 'feature_24', 'feature_63', 'feature_116', 'feature_32', 'feature_18'}


In [9]:
X.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_5',
       'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_19', 'feature_20', 'feature_21', 'feature_22',
       'feature_27', 'feature_29', 'feature_30', 'feature_31', 'feature_34',
       'feature_37', 'feature_39', 'feature_41', 'feature_42', 'feature_43',
       'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48',
       'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53',
       'feature_54', 'feature_55', 'feature_56', 'feature_59', 'feature_60',
       'feature_62', 'feature_64', 'feature_65', 'feature_67', 'feature_70',
       'feature_71', 'feature_72', 'feature_73', 'feature_74', 'feature_75',
       'feature_77', 'feature_78', 'feature_79', 'feature_80', 'feature_81',
       'feature_82', 'feature_83', 'feature_84', 'feature_85', 'feature_86',
       

In [10]:
del train

In [11]:
##train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)
#X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 0.1,random_state = 42)

In [12]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)


In [13]:
import xgboost
from xgboost import XGBClassifier,XGBRegressor
model = XGBClassifier(booster='gbtree', n_estimators=200, learning_rate=0.1, max_depth=20, use_label_encoder=False, tree_model='gpu_hist')
model.fit(X_train,y_train)


[16:00:22] WARNING: ../src/learner.cc:541: 
Parameters: { tree_model } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:02:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=2, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', tree_model='gpu_hist',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [14]:
yprd = model.predict(X_test)
model.score(X_test,y_test)

0.7007763215651847

In [15]:
model.save_model('./best.json')

In [16]:
test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')



In [17]:
test.drop('date', axis = 'columns',inplace = True)

In [18]:
feats = test[[str(col) for col in test.columns if 'feature' in col]]

In [19]:
X = pd.concat([remove_collinear_features(feats, 0.9), test['weight'], test['ts_id']], axis = 1)


Removed Columns {'feature_68', 'feature_126', 'feature_35', 'feature_34', 'feature_38', 'feature_128', 'feature_66', 'feature_28', 'feature_125', 'feature_10', 'feature_113', 'feature_71', 'feature_40', 'feature_107', 'feature_61', 'feature_23', 'feature_122', 'feature_108', 'feature_69', 'feature_36', 'feature_2', 'feature_16', 'feature_26', 'feature_57', 'feature_119', 'feature_4', 'feature_25', 'feature_49', 'feature_6', 'feature_14', 'feature_22', 'feature_20', 'feature_12', 'feature_8', 'feature_58', 'feature_114', 'feature_101', 'feature_30', 'feature_127', 'feature_129', 'feature_33', 'feature_124', 'feature_24', 'feature_63', 'feature_116', 'feature_32', 'feature_31', 'feature_18'}


In [20]:
testx = test.drop(columns=['feature_127', 'feature_26', 'feature_23', 'feature_35', 'feature_100', 'feature_32', 'feature_101', 'feature_125', 'feature_40', 'feature_88', 'feature_119', 'feature_129', 'feature_33', 'feature_124', 'feature_36', 'feature_76', 'feature_24', 'feature_116', 'feature_58', 'feature_68', 'feature_113', 'feature_6', 'feature_110', 'feature_128', 'feature_69', 'feature_122', 'feature_107', 'feature_126', 'feature_28', 'feature_57', 'feature_108', 'feature_25', 'feature_66', 'feature_38', 'feature_114', 'feature_18', 'feature_61', 'feature_63', 'feature_4'])

In [21]:
testx

,weight,feature_0,feature_1,feature_2,feature_3,feature_5,feature_7,feature_8,feature_9,feature_10,...,feature_109,feature_111,feature_112,feature_115,feature_117,feature_118,feature_120,feature_121,feature_123,ts_id
0,0.000000,1,-1.872746,-2.191242,-0.474163,0.014688,NaN,NaN,-0.989982,-1.055090,...,-0.892724,-0.156332,0.622816,2.561593,3.457757,6.649580,NaN,NaN,8.313583,0
1,16.673515,-1,-1.349537,-1.704709,0.068058,0.193794,NaN,NaN,-0.151877,-0.384952,...,-1.332492,-0.586619,-1.040491,0.983440,1.357907,1.612348,NaN,NaN,1.777472,1
2,0.000000,-1,0.812780,-0.256156,0.806463,-0.614188,NaN,NaN,5.448261,2.668029,...,1.193823,0.097345,0.796214,2.548596,0.882588,1.817895,NaN,NaN,9.667908,2
3,0.000000,-1,1.174378,0.344640,0.066872,-1.006373,NaN,NaN,4.508206,2.484260,...,2.513074,0.424964,1.992887,0.561528,-0.994041,0.099560,NaN,NaN,0.499251,3
4,0.138531,1,-3.172026,-3.093182,-0.161518,-0.195006,NaN,NaN,2.683018,1.450991,...,-0.934675,-0.373013,-1.213540,2.684119,2.861848,2.134804,NaN,NaN,4.101145,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15214,0.000000,1,-1.213240,-1.557117,0.530793,0.316811,0.741902,0.680989,0.501501,0.135505,...,1.641019,0.768819,1.917005,2.445685,1.666364,2.567707,0.169029,0.200094,2.551488,15214
15215,0.000000,1,-0.413328,-0.642504,0.429951,-0.728263,0.204294,0.138052,5.595514,4.562532,...,0.668457,0.570189,1.162564,5.046764,4.722535,5.901139,-0.551901,2.070960,22.159397,15215
15216,0.000000,1,-1.378947,-1.702976,0.548763,0.328203,0.784458,0.730435,0.816203,0.372920,...,2.225468,0.938165,2.364939,3.210722,1.956503,3.108164,0.169612,0.200728,2.685696,15216
15217,0.000000,1,-0.324708,-1.089962,-0.873900,-1.265208,2.302628,1.978776,5.817695,3.277066,...,0.635171,2.047227,1.136469,5.237434,11.283541,6.153315,-0.395420,0.300244,11.789678,15217


In [22]:
testx.columns

Index(['weight', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_5', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15',
       'feature_16', 'feature_17', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_27', 'feature_29', 'feature_30', 'feature_31',
       'feature_34', 'feature_37', 'feature_39', 'feature_41', 'feature_42',
       'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47',
       'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52',
       'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_59',
       'feature_60', 'feature_62', 'feature_64', 'feature_65', 'feature_67',
       'feature_70', 'feature_71', 'feature_72', 'feature_73', 'feature_74',
       'feature_75', 'feature_77', 'feature_78', 'feature_79', 'feature_80',
       'feature_81', 'feature_82', 'feature_83', 'feature_84', 'feature_85',
       'fea

In [23]:
ypred = model.predict(testx)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92'] ['weight', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_5', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_27', 'feature_29', 'feature_30

In [24]:
y_test = y[:len(test)]
y_test

sub = test[['ts_id']]
sub['action'] = yprd
sub.to_csv('xgb_trialsubmission', index = False)

ValueError: Length of values (398675) does not match length of index (15219)

In [25]:
import pickle
#load our saved model 
Pkl_Filename = "/kaggle/input/cat-model/cat_model.sav" 
with open(Pkl_Filename, 'rb') as file:  
    model = pickle.load(file)

model

import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df.action = model.predict(test_df) #make your 0/1 prediction here
    env.predict(sample_prediction_df) 

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/cat-model/cat_model.sav'

In [26]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import GridSearchCV
# model = KNeighborsClassifier()
# grid_params = {'n_neighbors':[5],
#               'weights':['uniform'],
#               'metric':['euclidean']}

In [27]:
# gs = GridSearchCV(model,grid_params)
# gs_results = gs.fit(X_train,y_train)
#gs_results.best_score_

In [28]:
# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()
# model.fit(X_train,y_train)

In [29]:
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(X_train,y_train)

In [30]:
# import xgboost
# from xgboost import XGBClassifier,XGBRegressor
# model = XGBClassifier()
# yprd = model.predict(X_test)
# model.score(X_test,y_test)

In [31]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# lda = LinearDiscriminantAnalysis(n_components = 5)
# X = lda.fit_transform(X_train,y_train)
# X_test = lda.transform(X_test)
#yprd = model.predict(X_test)
